# Simple measurament with 5 elements

# Overview

In order to simplify the coding there is a change of notation:
- The possible **states** will be labeled as 0, 1, ...
- The possible **results** also as 0, 1, ...

More useful definitions:
- An **experiment** is a serie of states. For example an experiment [0, 0, 0, 1, 1] means the system has produced the state 0 three times and then twice the state 1. 

In our case:
- 2 states are possible : {0,1}
- 2 results are possible : {0,1}
- All the experiments consists in 5 states, first state 0 and then state 1

Witha all those constraints, we can generate 5 possible experiments:
- $E_0$ = 0, 0, 0, 0, 0
- $E_1$ = 0, 0, 0, 0, 1
- $E_2$ = 0, 0, 0, 1, 1
- $E_3$ = 0, 0, 1, 1, 1
- $E_4$ = 0, 1, 1, 1, 1

The process is the following:
- Alice choses an experiment and send the states one by one to Bob
- Bob measure each state. Base on the results Bob guess which is the state associated using his **estimator**
- Once Bob has measured all the states generates an **hypothesis** of which was the experiment chosen by Alice based on the outputs of the measuraments he has performed
- The process is repeated several times and then the experiments are compared with the hypothesis to check the rate of success


# Probability of success : formula

Given an **experiment** $E_i$ we do a measurament getting a serie of results. The results obtained can be explained by a serie of **Hypothesis** and using our **estimator** we chose one of them, that has a certain probability of being the good. 

If we have K possible experiments, the probability of success is:
$$P_s = \sum_{i=1}^{M} {P(E_i)P(H_{chosen}|E_i)}$$
where:
- $P(E_i)$ : probability experiment $E_i$ is produced
- $P(H_{chosen}|E_i)$ : probability that the hypothesis we have chosen $H_{chosen}$ to explain the results produced by the experiment $E_i$ is the right one.

Ok, let's compute those probabilities in our scenario

$P(E_i)$ : Because we have 5 different experiments and all are the same likelihood, the probability is $\textbf{1/5}$

$P(H_{chosen}|E_i)$ : an experiment is a serie states $\Psi_i$ that we measure producing results $\Pi_j$; based on them we generate an hypothesis and the probabiliy of success is (where N is the number or states in the experiment): 

$$P(H_{chosen}|E_i) = \prod_{j=1}^{N}P({\Pi_{j; f(j)=i}|\Psi_i})$$ 

In our case, the estimator function is:
- $\Pi_1 \rightarrow \Psi_1$
- $\Pi_2 \rightarrow \Psi_2$

and the probabilities:
- $P(\Pi_1|\Psi_1) = 1$
- $P(\Pi_1|\Psi_2) = c^2$
- $P(\Pi_2|\Psi_1) = 0$
- $P(\Pi_2|\Psi_2) = 1 - c^2$

So if our experiments consists in $n_1$ states $\Psi_1$ and $n_2$ states $\Psi_2$ we can rewrite using our estimator:

$$P(H|E) = \prod_{i=1}^{n_1}{P(\Pi_1|\Psi_1)}\prod_{j=1}^{n_2}{P(\Pi_2|\Psi_2)} = (1-c^2)^{n_2}$$ 

In our case we have $M=5$ experiments, each of them consisting in $N=5$ possible states, where the value$n_2=[0,4]$. Putting all together we have:

$$P_s = {1 \over 5}((1-c^2)^0 + (1-c^2)^1 + (1-c^2)^2 + (1-c^2)^3 + (1-c^2)^4)$$

If we apply the formula for the geometric serie ($r \ne 1$):

${\displaystyle a+ar+ar^{2}+ar^{3}+\cdots +ar^{n}=\sum _{k=0}^{n}ar^{k}=a\left({\frac {1-r^{n+1}}{1-r}}\right),}$

we can write as

$$P_s = \frac{1-(1-c^2)^5}{1-c^2}$$



## Simulator


What follows is a program that emulates this process.

In [6]:
# -*- coding: utf-8 -*-

import argparse
import random
from datetime import datetime

random.seed(datetime.now())

# -----------------------------------------------------------------------------
# General Utilities
# -----------------------------------------------------------------------------

def _equal(array1, array2):
    """ Check the equality for two arrays. """

    if len(array1)!=len(array2):
        return False

    equal=True
    for ind, val1 in enumerate(array1):
        if val1!=array2[ind]:
            equal=False
            break

    return equal

The **estimator** is the function used by Bob to guess the state based on a result.

It is an array `estimator[<result>]=<state>` where:
- Index : the possible result
- Value : the state Bob associates to that result

For example estimator[a]=b means that if we get the reasult 'a' Bob will say that the state was 'b'.

From previous exercises, in our case this is the estimator:
- Result 0 $\rightarrow$ State 0
- Result 1 $\rightarrow$ State 1


In [8]:
def _getEstimator():
    return [0, 1]

The **probabiliy distribution** determines the probability that a Result is "produced" by a certain State.

It is a matrix `probDistribution[<state>][<result>] = <probability>` where:
- Row : states
- Column : results

For example `probDistribution[1][0]=0.25` means that the probability that **State 1** is measured as **Result 0** is of 0.25.

In our case c represents the scalar product between both states 
$$<State_1|State_0>=c$$

so the probability distribution genrated is the following:

In [7]:
def _getProbDistribution(c):
    return [[1   , 0],
            [c**2, 1-c**2]]

Now let's **build an experiment**. This is a serie of States given a certain distribution, where distribution is an array `distribution[<id>]=n` where `n` is the number times the state `<id>` appears in the experiment (order preserved).

For example [1,4] will generate [0,1,1,1,1]

In [9]:
def _buildExperiment(distribution):
    experiment=[]
    for ind, val in enumerate(distribution):
        experiment+=[ind]*val

    return experiment

Given a single **state** give a possible **result** taking into account the **probability distribution**

In [10]:
def _getResultForState(state, prob):
    value=random.uniform(0,1)

    output=None
    for ind,prob in enumerate(prob[state]):
        value-=prob
        if value<=0:
            output=ind
            break

    return output

Given an experiment, **give a possible list of results** taken into acount the **probability distribution**.

In [11]:
def _getResultsForExperiment(experiment, prob):
    results=[None] * len(experiment)

    for ind, state in enumerate(experiment):
        results[ind] = _getResultForState(state, prob)

    return results

Give a list of results, build an **hypothesis** (list of states) using the **estimator**.

In [12]:
def _getHypothesysForResults(results, estimator):
    """ Given a list of results, build an hypothesis using the estimator. """
    hypothesis=[None] * len(results)

    for ind, result in enumerate(results):
        hypothesis[ind] = estimator[result]

    return hypothesis

Ok, time to put all the pieces together.

In [21]:
# Generates a random string 
def computeSuccess(num_states_per_experiment, num_iterations_per_experiment, prob, estimator):

    tot=0
    totOk=0
    # If num_states_per_experiment = 5 we generates experiment of the form:
    # - 0 0 0 0 0
    # - 0 0 0 0 1
    # - 0 0 0 1 1
    # - 0 0 1 1 1
    # - 0 1 1 1 1
    # And every experiment is generated 'num_iterations_per_experiment' times
    # n is the number of states '
    for n in range(num_states_per_experiment):
        experiment=_buildExperiment([num_states_per_experiment-n,n])
        myOk=0
        myTot=0
        for k in range(num_iterations_per_experiment):
            results=_getResultsForExperiment(experiment, prob)
            hypothesis=_getHypothesysForResults(results, estimator)
            tot += 1
            myTot += 1
            if _equal(experiment, hypothesis):
                totOk += 1
                myOk += 1

    return (totOk * 100.0) / tot

# Some computations

Let's compute several cases

**Value c=0 (totalment distinguished)**

In [25]:
n=5
c=0.00000001
i=100

theoretical=100.0 * ((1-(1-c**2)**n)/(1-(1-c**2)))/n
simulation = computeSuccess(n, i, _getProbDistribution(c), _getEstimator())

print ("Theoretical : %f" % (theoretical))
print ("Simulation : %f" % (simulation))

Theoretical : 100.000000
Simulation : 100.000000


**Value c=1 (can not be distiguished)**

In [27]:
n=5
c=1
i=100

theoretical=100.0 * ((1-(1-c**2)**n)/(1-(1-c**2)))/n
simulation = computeSuccess(n, i, _getProbDistribution(c), _getEstimator())

print ("Theoretical : %f" % (theoretical))
print ("Simulation : %f" % (simulation))

Theoretical : 20.000000
Simulation : 20.000000


**Intermediate value**

In [28]:
n=5
c=0.5
i=100

theoretical=100.0 * ((1-(1-c**2)**n)/(1-(1-c**2)))/n
simulation = computeSuccess(n, i, _getProbDistribution(c), _getEstimator())

print ("Theoretical : %f" % (theoretical))
print ("Simulation : %f" % (simulation))

Theoretical : 61.015625
Simulation : 61.800000
